In [1]:
import coiled

In [2]:
coiled.Cloud()

<Cloud: adbreind@https://beta.coiled.io - running>

In [3]:
cluster = coiled.Cluster(n_workers=4)

Creating Cluster. This takes about a minute ...Checking environment images
Valid environment image found


In [7]:
from dask.distributed import Client
client = Client(cluster)
print("Dashboard:", client.dashboard_link)

Dashboard: http://ec2-52-15-98-174.us-east-2.compute.amazonaws.com:8787/status


/Users/adam/anaconda3/envs/dask2/lib/python3.8/site-packages/distributed/client.py:1136: VersionMismatchWarning: Mismatched versions found

+-------------+--------+-----------+---------+
| Package     | client | scheduler | workers |
+-------------+--------+-----------+---------+
| cloudpickle | 1.5.0  | 1.6.0     | 1.6.0   |
| distributed | 2.23.0 | 2.24.0    | 2.24.0  |
+-------------+--------+-----------+---------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


In [8]:
import s3fs
fs = s3fs.S3FileSystem(anon=True)
trips = fs.ls('nyc-tlc/trip data', detail=True)

In [9]:
files = ['s3://' + trip['Key'] for trip in trips if ('yellow_tripdata_2019' in trip['Key'])]

In [22]:
from dask import dataframe as dd

df = dd.read_csv(
    files[:2],
    dtype={
        "payment_type": "UInt8",
        "VendorID": "UInt8",
        "passenger_count": "UInt8",
        "RatecodeID": "UInt8",
    },
    storage_options={"anon": True},
)

In [23]:
df.npartitions

22

In [24]:
floatcols = [c for c in df.columns if df[c].dtype == 'float64']
floatcols

['trip_distance',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'congestion_surcharge']

In [119]:
arr = df[floatcols].dropna().to_dask_array(lengths=True).rechunk(chunks=(310_000,9)).persist()
arr

dask.array<rechunk-merge, shape=(9831189, 9), dtype=float64, chunksize=(310000, 9), chunktype=numpy.ndarray>

In [120]:
import dask.array as da

In [121]:
u, s, v = da.linalg.svd(arr)

In [122]:
s.compute()

array([1.08565697e+06, 2.20468046e+04, 8.28676442e+03, 5.64054862e+03,
       4.49733828e+03, 3.01440914e+03, 1.18977887e+03, 4.71922894e+02,
       5.97750968e+01])

In [126]:
%%timeit

s.compute()

distributed.client - WARNING - Couldn't gather 1 keys, rescheduling {"('getitem-1bd6e4e75789cd86e50f1e83adc42880-s2', 0)": ()}


The slowest run took 7.62 times longer than the fastest. This could mean that an intermediate result is being cached.
194 ms ± 219 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Ranges from 100ms to 500ms, with typical speed about ~200ms on 16 cores (4 nodes) on a clean environment

In [128]:
# client.restart()

Client Scheduler: tls://ec2-52-15-98-174.us-east-2.compute.amazonaws.com:8786 Dashboard: http://ec2-52-15-98-174.us-east-2.compute.amazonaws.com:8787/status,Cluster Workers: 4 Cores: 16 Memory: 68.72 GB


In [129]:
client.close()

In [130]:
cluster.close()